In [117]:
import glob
import os
import re
import sys
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
# import lightgbm as lgb

warnings.filterwarnings('ignore')

In [118]:
train = pd.read_csv('../processed_data/train_v7.csv')
test = pd.read_csv('../processed_data/test_v7.csv')

In [65]:
len(train),len(test)

(31467, 31262)

In [72]:
train['location']

0              東京都北区滝野川３丁目
1              東京都中央区月島３丁目
2              東京都渋谷区笹塚２丁目
3        東京都杉並区高円寺南２丁目23-2
4           東京都葛飾区金町３丁目7-2
               ...        
31462          東京都板橋区蓮根２丁目
31463    東京都世田谷区太子堂５丁目17-1
31464          東京都江東区南砂４丁目
31465          東京都中野区中野２丁目
31466          東京都豊島区千川２丁目
Name: location, Length: 31467, dtype: object

In [73]:
train['location'].str.extract('(.+)都(.*区)(.*)',expand=True)

,0,1,2
0,東京,北区,滝野川３丁目
1,東京,中央区,月島３丁目
2,東京,渋谷区,笹塚２丁目
3,東京,杉並区,高円寺南２丁目23-2
4,東京,葛飾区,金町３丁目7-2
...,...,...,...
31462,東京,板橋区,蓮根２丁目
31463,東京,世田谷区,太子堂５丁目17-1
31464,東京,江東区,南砂４丁目
31465,東京,中野区,中野２丁目


In [74]:
layout = pd.concat([train['layout'],test['layout']])

In [75]:
layout.value_counts()

1K            23661
1R            10177
1LDK           9702
2LDK           5130
1DK            3781
2DK            3715
3LDK           2240
2K             1807
3DK             851
1LDK+S(納戸)      487
2LDK+S(納戸)      326
4LDK            250
3LDK+S(納戸)      133
1K+S(納戸)        130
3K              102
5LDK             38
2DK+S(納戸)        37
4DK              33
1DK+S(納戸)        27
4LDK+S(納戸)       24
4K               12
3DK+S(納戸)        10
6LDK              9
5LDK+S(納戸)        9
5DK               8
5K                7
6LDK+S(納戸)        4
2K+S(納戸)          3
5DK+S(納戸)         3
4DK+S(納戸)         2
6DK               2
1LK+S(納戸)         2
3K+S(納戸)          2
8LDK              1
2R                1
6K                1
11R               1
1LK               1
Name: layout, dtype: int64

In [76]:
test[test['layout']=='11R']

,id,location,access,layout,age,direction,area,floor,bath_toilet,kitchen,...,23ku_countall,area_num_countall,age_countall,floor_countall,max_floor_countall,layout_countall,room_num_countall,direction_countall,facilities_countall,contact_period_countall
8483,39954,東京都世田谷区桜上水５丁目,京王線\t桜上水駅\t徒歩5分\t\t京王線\t上北沢駅\t徒歩10分\t\t東急世田谷線\...,11R,64年0ヶ月,南,16m2,1,専用バス／\t専用トイレ／\tシャワー,ガスコンロ／\t冷蔵庫あり,...,5971,262,1,12112,12311,1,47969,17695,54,45052


In [77]:
test[test['layout']=='2R']

,id,location,access,layout,age,direction,area,floor,bath_toilet,kitchen,...,23ku_countall,area_num_countall,age_countall,floor_countall,max_floor_countall,layout_countall,room_num_countall,direction_countall,facilities_countall,contact_period_countall
24546,56017,東京都北区志茂２丁目,京浜東北線\t赤羽駅\t徒歩10分\t\t南北線\t志茂駅\t徒歩4分\t\t南北線\t赤羽...,2R,55年10ヶ月,南,14.8m2,2,共同トイレ,NaN,...,2061,8,7,17201,12311,1,11019,17695,1,45052


In [78]:
train.columns

Index(['id', 'y', 'location', 'access', 'layout', 'age', 'direction', 'area',
       'floor', 'bath_toilet', 'kitchen', 'broadcast_com', 'facilities',
       'parking', 'enviroment', 'structure', 'contact_period', 'log_y',
       'area_num', 'age_month', 'max_floor', 'room_num', 'L', 'D', 'K', 'R',
       'S', 'north', 'structure_orderd', 'walk_time', 'min_time', 'avg_time',
       'bicycle_parking', 'car_parking', 'bike_parking', 'toilet', 'bath',
       'sm_doku', 'kanso', 'onsui', 'oidaki', 'b_t_split', '23ku',
       '23ku_mean_std', '23ku_countall', 'area_num_countall', 'age_countall',
       'floor_countall', 'max_floor_countall', 'layout_countall',
       'room_num_countall', 'direction_countall', 'facilities_countall',
       'contact_period_countall'],
      dtype='object')

In [79]:
c_period = pd.concat([train['contact_period'],test['contact_period']])

In [80]:
c_period.value_counts()

2年間                              45052
2年間\t※この物件は\t定期借家\tです。            1888
5年間\t※この物件は\t定期借家\tです。             506
3年間\t※この物件は\t定期借家\tです。             443
1年間                                163
                                 ...  
12年間                                 1
2025年8月まで\t※この物件は\t定期借家\tです。         1
2021年1月まで\t※この物件は\t定期借家\tです。         1
3年9ヶ月間\t※この物件は\t定期借家\tです。            1
2020年11月まで\t※この物件は\t定期借家\tです。        1
Name: contact_period, Length: 79, dtype: int64

In [81]:
def teiki_syakuya(c_period):
    res = []
    for c in c_period.fillna(''):
        if '定期借家' in c:
            res.append(1)
        else:
            res.append(0)
    return res

In [82]:
c_period.isnull().sum()

14098

In [103]:
train_v7 = pd.read_csv('../processed_data/train_v7.csv')
# train_v7.isnull().sum()

In [105]:
train_v7.isnull().sum()

id                            0
y                             0
location                      0
access                        0
layout                        0
age                           0
direction                  2749
area                          0
floor                         0
bath_toilet                 329
kitchen                     621
broadcast_com              3440
facilities                   82
parking                    5285
enviroment                 9430
structure                     0
contact_period             7117
log_y                         0
area_num                      0
age_month                     0
max_floor                     0
room_num                      0
L                             0
D                             0
K                             0
R                             0
S                             0
north                         0
structure_orderd              0
walk_time                     0
min_time                      0
avg_time

In [102]:
train_v7['parking'].value_counts()

駐車場\t無                                           3269
駐輪場\t空有\t駐車場\t無\tバイク置き場\t無                       2374
駐車場\t無\t駐輪場\t無\tバイク置き場\t無                        1832
駐輪場\t空有                                          1246
駐輪場\t空有\tバイク置き場\t無                                957
                                                 ... 
駐車場\t空有\t23,000円\t(※空き要確認)                          1
駐車場\t空無\t36,000円(税込)                                1
駐車場\t空有\t32,400円\t駐輪場\t空有\t0円                       1
駐車場\t空有(6台)\t32,400円                                1
駐車場\t空有\t14,000円\t駐輪場\t空有\t0円\tバイク置き場\t空有\t0円       1
Name: parking, Length: 3911, dtype: int64

In [106]:
train_v7['bath_toilet'].value_counts()

専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t追焚機能／\t浴室乾燥機\t／\t温水洗浄便座／\t洗面台独立／\t脱衣所    4606
専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t浴室乾燥機\t／\t温水洗浄便座／\t洗面台独立／\t脱衣所           2936
専用バス／\t専用トイレ／\tシャワー                                                      2133
専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー                                            1495
専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t追焚機能／\t浴室乾燥機\t／\t温水洗浄便座／\t洗面台独立           873
                                                                         ... 
バスなし                                                                        1
専用バス／\tシャワー／\t追焚機能                                                          1
専用バス／\t専用トイレ／\tシャワー／\t温水洗浄便座／\t脱衣所                                          1
共同バス／\t共同トイレ／\tバス・トイレ別／\tシャワー                                               1
専用バス／\t専用トイレ／\tシャワー／\t追焚機能／\t温水洗浄便座／\t脱衣所                                   1
Name: bath_toilet, Length: 360, dtype: int64

In [107]:
kitchen = pd.concat([train['kitchen'],test['kitchen']])
kitchen

0              ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t給湯
1                      ガスコンロ／\tシステムキッチン\t／\t給湯
2              ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t給湯
3                                   ガスコンロ／\t給湯
4                                  給湯／\t独立キッチン
                         ...                  
31257          ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t給湯
31258      ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t独立キッチン
31259                       ガスコンロ／\tコンロ3口／\t給湯
31260    IHコンロ\t／\tコンロ3口／\tシステムキッチン\t／\t独立キッチン
31261                  コンロ2口／\tシステムキッチン\t／\t給湯
Name: kitchen, Length: 62729, dtype: object

In [108]:
kitchen.unique()

array(['ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t給湯', 'ガスコンロ／\tシステムキッチン\t／\t給湯',
       'ガスコンロ／\t給湯', '給湯／\t独立キッチン',
       'ガスコンロ／\tコンロ3口／\tシステムキッチン\t／\t給湯／\t独立キッチン',
       'ガスコンロ／\tコンロ2口／\tシステムキッチン', 'ガスコンロ／\tコンロ3口／\tシステムキッチン\t／\t給湯',
       'ガスコンロ／\tコンロ2口／\t給湯', nan, 'コンロ2口／\t給湯', '給湯',
       'IHコンロ\t／\tコンロ3口／\tシステムキッチン',
       'IHコンロ\t／\tコンロ3口／\tシステムキッチン\t／\t独立キッチン', 'ガスコンロ／\tコンロ1口／\t給湯',
       'ガスコンロ／\tコンロ1口／\t給湯／\t冷蔵庫あり',
       'ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t給湯／\t独立キッチン',
       'コンロ3口／\tシステムキッチン\t／\t給湯', 'IHコンロ\t／\tシステムキッチン\t／\t給湯',
       'コンロ設置可（コンロ2口）',
       'ガスコンロ／\tコンロ3口／\tシステムキッチン\t／\tカウンターキッチン\t／\t給湯／\tL字キッチン',
       'ガスコンロ／\tコンロ1口／\t給湯／\t独立キッチン',
       'ガスコンロ／\tコンロ3口／\tシステムキッチン\t／\tカウンターキッチン\t／\t給湯',
       'ガスコンロ／\tコンロ設置可（口数不明）／\t給湯',
       'ガスコンロ／\tコンロ2口／\t給湯／\t独立キッチン／\tL字キッチン', 'システムキッチン',
       'ガスコンロ／\tコンロ2口', '電気コンロ／\t給湯',
       'IHコンロ\t／\tコンロ2口／\tシステムキッチン\t／\t給湯／\tL字キッチン',
       'コンロ2口／\tシステムキッチン\t／\t給湯', 'コンロ2口／\tシステムキッチン\t／\t独立キッチン',
       'ガスコンロ／\tコンロ2口／\tシステム

In [110]:
train[train['kitchen'].isnull()].loc[:,['kitchen','layout']]

,kitchen,layout
11,NaN,1K
73,NaN,1LDK
74,NaN,1R
134,NaN,1K
139,NaN,1K
...,...,...
31008,NaN,1R
31027,NaN,1R
31075,NaN,1R
31155,NaN,2K


In [112]:
hoge = []
for k in kitchen.fillna(''):
    split_k = k.split('\t')
    hoge+=split_k

In [113]:
set(hoge)

{'',
 'IHコンロ',
 'L字キッチン',
 'カウンターキッチン',
 'ガスコンロ',
 'ガスコンロ／',
 'コンロ1口',
 'コンロ1口／',
 'コンロ2口',
 'コンロ2口／',
 'コンロ3口',
 'コンロ3口／',
 'コンロ4口以上',
 'コンロ4口以上／',
 'コンロ設置可（コンロ1口）',
 'コンロ設置可（コンロ1口）／',
 'コンロ設置可（コンロ2口）',
 'コンロ設置可（コンロ2口）／',
 'コンロ設置可（コンロ3口）',
 'コンロ設置可（コンロ3口）／',
 'コンロ設置可（コンロ4口以上）／',
 'コンロ設置可（口数不明）',
 'コンロ設置可（口数不明）／',
 'システムキッチン',
 '冷蔵庫あり',
 '冷蔵庫あり／',
 '独立キッチン',
 '独立キッチン／',
 '給湯',
 '給湯／',
 '電気コンロ',
 '電気コンロ／',
 '／'}

In [114]:
train.groupby('kitchen').describe()['y']

,count,mean,std,min,25%,50%,75%,max
kitchen,,,,,,,,
IHコンロ,110.0,67331.818182,24792.651811,35000.0,53000.0,62000.0,70000.0,175000.0
IHコンロ\t／\tカウンターキッチン\t／\t給湯,10.0,124750.000000,67242.203183,62000.0,73750.0,113750.0,147000.0,280000.0
IHコンロ\t／\tコンロ1口,79.0,71602.531646,34557.124668,28000.0,58000.0,65000.0,78400.0,334800.0
IHコンロ\t／\tコンロ1口／\tカウンターキッチン,1.0,64000.000000,NaN,64000.0,64000.0,64000.0,64000.0,64000.0
IHコンロ\t／\tコンロ1口／\tカウンターキッチン\t／\t給湯,5.0,102200.000000,41166.734143,76000.0,79000.0,79000.0,104000.0,173000.0
...,...,...,...,...,...,...,...,...
電気コンロ／\tシステムキッチン\t／\t給湯／\t冷蔵庫あり,3.0,64000.000000,1732.050808,63000.0,63000.0,63000.0,64500.0,66000.0
電気コンロ／\tシステムキッチン\t／\t給湯／\t独立キッチン,1.0,76000.000000,NaN,76000.0,76000.0,76000.0,76000.0,76000.0
電気コンロ／\t給湯,61.0,62947.540984,10804.838524,41000.0,55000.0,61000.0,70000.0,93000.0


In [115]:
train.groupby('facilities').describe()['y']

,count,mean,std,min,25%,50%,75%,max
facilities,,,,,,,,
24時間換気システム\t／\t室内洗濯機置場／\tタイル張り\t公営水道／\t下水／\t都市ガス,2.0,64500.000000,707.106781,64000.0,64250.0,64500.0,64750.0,65000.0
2面採光\t公営水道／\t下水／\tプロパンガス,1.0,62000.000000,NaN,62000.0,62000.0,62000.0,62000.0,62000.0
2面採光\t公営水道／\t下水／\t都市ガス,5.0,53040.000000,9513.569257,46000.0,46000.0,48200.0,57000.0,68000.0
2面採光\t都市ガス,1.0,80000.000000,NaN,80000.0,80000.0,80000.0,80000.0,80000.0
2面採光／\tクッションフロア／\t室内洗濯機置場\t公営水道／\t下水／\t都市ガス,1.0,48000.000000,NaN,48000.0,48000.0,48000.0,48000.0,48000.0
...,...,...,...,...,...,...,...,...
水道その他／\t下水／\t都市ガス,3.0,38733.333333,3002.221400,37000.0,37000.0,37000.0,39600.0,42200.0
洗濯機置場なし\t公営水道／\t下水／\t都市ガス,2.0,46000.000000,8485.281374,40000.0,43000.0,46000.0,49000.0,52000.0
洗濯機置場なし\t公営水道／\t都市ガス,1.0,40000.000000,NaN,40000.0,40000.0,40000.0,40000.0,40000.0


In [116]:
train.groupby('broadcast_com').describe()['y']

,count,mean,std,min,25%,50%,75%,max
broadcast_com,,,,,,,,
BSアンテナ,315.0,107991.587302,40671.573752,29500.0,82750.0,101000.0,125250.0,350000.0
BSアンテナ／\tインターネット使用料無料,27.0,100814.814815,15417.989086,65000.0,92250.0,100500.0,111000.0,143500.0
CATV,979.0,100575.464760,80371.396048,33000.0,66000.0,82100.0,110000.0,1660000.0
CATV／\tBSアンテナ,216.0,122553.175926,92142.335408,48000.0,79000.0,98000.0,133500.0,730000.0
CATV／\tBSアンテナ／\tインターネット使用料無料,6.0,85250.000000,27777.238884,55000.0,70875.0,73000.0,107000.0,122000.0
...,...,...,...,...,...,...,...,...
高速インターネット／\t光ファイバー／\tCATV,88.0,109873.863636,79254.167727,45000.0,71000.0,90750.0,120000.0,598000.0
高速インターネット／\t光ファイバー／\tCATV／\tBSアンテナ,25.0,98160.000000,45456.646012,53000.0,78500.0,80000.0,105000.0,256000.0
高速インターネット／\t光ファイバー／\tCATV／\tCSアンテナ,6.0,249500.000000,106893.872603,45000.0,245000.0,276000.0,313000.0,340000.0


### kitchenの前処理

In [4]:
kitchen = pd.concat([train['kitchen'],test['kitchen']])
kitchen

0              ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t給湯
1                      ガスコンロ／\tシステムキッチン\t／\t給湯
2              ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t給湯
3                                   ガスコンロ／\t給湯
4                                  給湯／\t独立キッチン
                         ...                  
31257          ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t給湯
31258      ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t独立キッチン
31259                       ガスコンロ／\tコンロ3口／\t給湯
31260    IHコンロ\t／\tコンロ3口／\tシステムキッチン\t／\t独立キッチン
31261                  コンロ2口／\tシステムキッチン\t／\t給湯
Name: kitchen, Length: 62729, dtype: object

In [28]:
hoge = []
for k in kitchen.fillna(''):
    k_split = k.split('\t')
    hoge += k_split

In [29]:
k_uni = set(hoge)
k_uni = sorted(list(k_uni))
k_uni

['',
 'IHコンロ',
 'L字キッチン',
 'カウンターキッチン',
 'ガスコンロ',
 'ガスコンロ／',
 'コンロ1口',
 'コンロ1口／',
 'コンロ2口',
 'コンロ2口／',
 'コンロ3口',
 'コンロ3口／',
 'コンロ4口以上',
 'コンロ4口以上／',
 'コンロ設置可（コンロ1口）',
 'コンロ設置可（コンロ1口）／',
 'コンロ設置可（コンロ2口）',
 'コンロ設置可（コンロ2口）／',
 'コンロ設置可（コンロ3口）',
 'コンロ設置可（コンロ3口）／',
 'コンロ設置可（コンロ4口以上）／',
 'コンロ設置可（口数不明）',
 'コンロ設置可（口数不明）／',
 'システムキッチン',
 '冷蔵庫あり',
 '冷蔵庫あり／',
 '独立キッチン',
 '独立キッチン／',
 '給湯',
 '給湯／',
 '電気コンロ',
 '電気コンロ／',
 '／']

In [51]:
rm_idx = []
for i,u in enumerate(k_uni):
    if '／' in u:
        rm_idx.append(i)

In [53]:
k_uni = [k_uni[i] for i in range(len(k_uni)) if i not in rm_idx]

In [54]:
k_uni

['',
 'IHコンロ',
 'L字キッチン',
 'カウンターキッチン',
 'ガスコンロ',
 'コンロ1口',
 'コンロ2口',
 'コンロ3口',
 'コンロ4口以上',
 'コンロ設置可（コンロ1口）',
 'コンロ設置可（コンロ2口）',
 'コンロ設置可（コンロ3口）',
 'コンロ設置可（口数不明）',
 'システムキッチン',
 '冷蔵庫あり',
 '独立キッチン',
 '給湯',
 '電気コンロ']

In [91]:
df = pd.concat([train,test])
for i,v in enumerate(k_uni[1:]):
    l = []
#     print(v)
    for k in kitchen.fillna(' '):
        if v in k:
#             print(k)
            l.append(1)
        else:
            l.append(0)
#     print(len(l))
    df['kitchen_'+str(i)] = l

In [92]:
df.iloc[:,-1*len(k_uni):]

,y,kitchen_0,kitchen_1,kitchen_2,kitchen_3,kitchen_4,kitchen_5,kitchen_6,kitchen_7,kitchen_8,kitchen_9,kitchen_10,kitchen_11,kitchen_12,kitchen_13,kitchen_14,kitchen_15,kitchen_16
0,75000.0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0
1,76000.0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0
2,110000.0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0
3,150000.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
4,74000.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31257,NaN,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0
31258,NaN,0,0,0,1,0,1,0,0,0,0,0,0,1,0,1,0,0
31259,NaN,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0
31260,NaN,1,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0


In [94]:
df[df['kitchen_0']==1]['kitchen']

18                            IHコンロ\t／\tコンロ3口／\tシステムキッチン
22                 IHコンロ\t／\tコンロ3口／\tシステムキッチン\t／\t独立キッチン
30                             IHコンロ\t／\tシステムキッチン\t／\t給湯
51            IHコンロ\t／\tコンロ2口／\tシステムキッチン\t／\t給湯／\tL字キッチン
77                                  IHコンロ\t／\tコンロ1口／\t給湯
                              ...                       
31205                               IHコンロ\t／\tコンロ1口／\t給湯
31208                  IHコンロ\t／\tコンロ2口／\tシステムキッチン\t／\t給湯
31222    IHコンロ\t／\tコンロ3口／\tシステムキッチン\t／\tカウンターキッチン\t／\t給湯
31256                                 IHコンロ\t／\tシステムキッチン
31260              IHコンロ\t／\tコンロ3口／\tシステムキッチン\t／\t独立キッチン
Name: kitchen, Length: 8851, dtype: object

In [99]:
def processing_kitchen(train,test):
    kitchen = pd.concat([train['kitchen'],test['kitchen']])
    
    k_uni = []
    for k in kitchen.fillna(''):
        k_split = k.split('\t')
        k_uni += k_split
        
    k_uni = sorted(list(set(k_uni)))
    
    rm_idx = []
    for i,u in enumerate(k_uni):
        if '／' in u:
            rm_idx.append(i)
            
    k_uni = [k_uni[i] for i in range(len(k_uni)) if i not in rm_idx]    
    n_df = []
    for df in [train,test]:
        for i,v in enumerate(k_uni[1:]):
            l = []

            for k in df['kitchen'].fillna(' '):
                if v in k:
                    l.append(1)
                else:
                    l.append(0)
            df['kitchen_'+str(i)] = l
            
        n_df.append(df)
        
    return n_df[0],n_df[1]

In [100]:
tr, te = processing_kitchen(train,test)

In [102]:
train.columns

Index(['id', 'y', 'location', 'access', 'layout', 'age', 'direction', 'area',
       'floor', 'bath_toilet', 'kitchen', 'broadcast_com', 'facilities',
       'parking', 'enviroment', 'structure', 'contract_period', 'log_y',
       'area_num', 'age_month', 'max_floor', 'room_num', 'L', 'D', 'K', 'R',
       'S', 'north', 'structure_orderd', 'walk_time', 'min_time', 'avg_time',
       'bicycle_parking', 'car_parking', 'bike_parking', 'toilet', 'bath',
       'sm_doku', 'kanso', 'onsui', 'oidaki', 'b_t_split', 'teiki_syakuya',
       '23ku', '23ku_mean_std', '23ku_countall', 'area_num_countall',
       'age_countall', 'floor_countall', 'max_floor_countall',
       'layout_countall', 'room_num_countall', 'direction_countall',
       'facilities_countall', 'contract_period_countall', 'kitchen_0',
       'kitchen_1', 'kitchen_2', 'kitchen_3', 'kitchen_4', 'kitchen_5',
       'kitchen_6', 'kitchen_7', 'kitchen_8', 'kitchen_9', 'kitchen_10',
       'kitchen_11', 'kitchen_12', 'kitchen_13', 'ki

In [114]:
def processing_broadcast_com(train,test):
    broadcast_com = pd.concat([train['broadcast_com'],test['broadcast_com']])
    
    b_uni = []
    for b in broadcast_com.fillna(''):
        b_split = b.split('\t')
        b_uni += b_split
        
    b_uni = sorted(list(set(b_uni)))
    
    rm_idx = []
    for i,u in enumerate(b_uni):
        if '／' in u:
            rm_idx.append(i)
            
    b_uni = [b_uni[i] for i in range(len(b_uni)) if i not in rm_idx]
#     print(b_uni)
    
    n_df = []
    for df in [train,test]:
        for i,v in enumerate(b_uni[1:]):
            l = []

            for k in df['broadcast_com'].fillna(' '):
                if v in k:
                    l.append(1)
                else:
                    l.append(0)
            df['bc_'+str(i)] = l
            
        n_df.append(df)
        
    return n_df[0],n_df[1]

In [115]:
tr, te = processing_broadcast_com(train,test)

['', 'BSアンテナ', 'CATV', 'CSアンテナ', 'インターネット使用料無料', 'インターネット対応', '光ファイバー', '有線放送', '高速インターネット']


In [116]:
tr.loc[:,'broadcast_com':]

,broadcast_com,facilities,parking,enviroment,structure,contract_period,log_y,area_num,age_month,max_floor,...,kitchen_17,bc_0,bc_1,bc_2,bc_3,bc_4,bc_5,bc_6,bc_7,bc_8
0,インターネット対応／\tCATV／\tCSアンテナ／\tBSアンテナ,エアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t室内洗濯機置場／\t敷...,駐輪場\t空有,【小学校】 495m\t【大学】 461m\t【小学校】 962m\t【公園】 1103m\...,RC（鉄筋コンクリート）,1,11.225243,20.01,117,12,...,0,1,1,1,0,1,0,0,0,0
1,インターネット対応,エアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t室内洗濯機置場／\t敷...,駐輪場\t空有\t駐車場\t無\tバイク置き場\t無,【スーパー】 1283m,鉄骨造,2,11.238489,16.50,538,10,...,0,0,0,0,0,1,0,0,0,0
2,インターネット対応／\t光ファイバー／\tCSアンテナ／\tBSアンテナ,エアコン付\tウォークインクローゼット\tシューズボックス／\tバルコニー／\tフローリング...,"駐輪場\t空有\tバイク置き場\t空有\t駐車場\t近隣\t30,000円(税込)\t距離100m",【スーパー】 89m\t【コンビニ】 184m\t【コンビニ】 392m\t【スーパー】 492m,RC（鉄筋コンクリート）,3,11.608236,22.05,102,15,...,0,1,0,1,0,1,1,0,0,0
3,インターネット対応／\t光ファイバー,エアコン付\tシューズボックス／\tバルコニー／\t2面採光／\t室内洗濯機置場／\tエレベ...,駐車場\t無\t駐輪場\t無\tバイク置き場\t無,【スーパー】 225m\t【スーパー】 448m\t【スーパー】 619m\t【スーパー】 ...,RC（鉄筋コンクリート）,4,11.918391,60.48,352,4,...,0,0,0,0,0,1,1,0,0,0
4,NaN,バルコニー／\tフローリング／\t室内洗濯機置場\t公営水道／\t下水,"駐車場\t近隣\t17,000円(税込)\t距離300m\t駐輪場\t無\tバイク置き場\t無",【スーパー】 193m\t【スーパー】 298m\t【スーパー】 660m\t【スーパー】 ...,木造,5,11.211820,39.66,379,2,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31462,インターネット対応,エアコン付\tバルコニー／\t室内洗濯機置場\t公営水道／\t都市ガス,駐車場\t無,NaN,鉄骨造,31466,11.289782,37.90,360,3,...,0,0,0,0,0,1,0,0,0,0
31463,NaN,エアコン付\tバルコニー／\tフローリング／\t室外洗濯機置場／\tタイル張り\t公営水道／...,駐車場\t無\t駐輪場\t無\tバイク置き場\t無,【スーパー】 458m\t【スーパー】 540m\t【コンビニ】 131m\t【コンビニ】 ...,RC（鉄筋コンクリート）,31467,10.905956,12.04,475,4,...,0,0,0,0,0,0,0,0,0,0
31464,光ファイバー,シューズボックス／\tバルコニー／\tフローリング／\t室内洗濯機置場\t都市ガス,駐輪場\t空有\tバイク置き場\t無,【コンビニ】 302m\t【コンビニ】 394m\t【コンビニ】 452m\t【コンビニ】 ...,鉄骨造,31468,11.736069,60.00,550,5,...,0,0,0,0,0,0,1,0,0,0
31465,インターネット対応／\t光ファイバー／\tCATV,エアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t3面採光／\t室内洗濯...,駐輪場\t空有\t駐車場\t無\tバイク置き場\t無,【スーパー】 485m\t【スーパー】 1051m\t【コンビニ】 476m\t【コンビニ】...,木造,31469,11.492723,29.59,132,2,...,0,0,1,0,0,1,1,0,0,0


In [119]:
def processing_facilities(train,test):
    facilities = pd.concat([train['facilities'],test['facilities']])
    
    f_uni = []
    for f in facilities.fillna(''):
        f_split = f.split('\t')
        f_uni += f_split
        
    f_uni = sorted(list(set(f_uni)))
    
    rm_idx = []
    for i,u in enumerate(f_uni):
        if '／' in u:
            rm_idx.append(i)
            
    f_uni = [f_uni[i] for i in range(len(f_uni)) if i not in rm_idx]
    print(f_uni)
    
    n_df = []
    for df in [train,test]:
        for i,v in enumerate(f_uni[1:]):
            l = []

            for k in df['facilities'].fillna(' '):
                if v in k:
                    l.append(1)
                else:
                    l.append(0)
            df['facilities_'+str(i)] = l
            
        n_df.append(df)
        
    return n_df[0],n_df[1]

In [121]:
tr,te = processing_facilities(train,test)

['', '24時間換気システム', '2面採光', '3面採光', 'ウォークインクローゼット', 'エアコン付', 'エレベーター', 'オール電化', 'ガスその他', 'ガス暖房', 'クッションフロア', 'シューズボックス', 'タイル張り', 'トランクルーム', 'バリアフリー', 'バルコニー', 'フローリング', 'プロパンガス', 'ペアガラス', 'ルーフバルコニー', 'ロフト付き', '下水', '二世帯住宅', '二重サッシ', '公営水道', '冷房', '出窓', '室内洗濯機置場', '室外洗濯機置場', '専用庭', '床下収納', '床暖房', '排水その他', '敷地内ごみ置き場', '水道その他', '洗濯機置場なし', '都市ガス', '防音室']


In [122]:
tr

,id,y,location,access,layout,age,direction,area,floor,bath_toilet,...,facilities_27,facilities_28,facilities_29,facilities_30,facilities_31,facilities_32,facilities_33,facilities_34,facilities_35,facilities_36
0,1,75000,東京都北区滝野川３丁目,都営三田線\t西巣鴨駅\t徒歩4分\t\t埼京線\t板橋駅\t徒歩14分\t\t都電荒川線\...,1K,9年9ヶ月,南東,20.01m2,1,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t浴室乾燥機\t／\t温水洗浄便座,...,0,0,0,0,0,1,0,0,1,0
1,2,76000,東京都中央区月島３丁目,都営大江戸線\t勝どき駅\t徒歩5分\t\t有楽町線\t月島駅\t徒歩9分\t\t日比谷線\...,1R,44年10ヶ月,NaN,16.5m2,5,専用トイレ／\tシャワー／\t温水洗浄便座,...,0,0,0,0,0,1,0,0,0,0
2,3,110000,東京都渋谷区笹塚２丁目,京王線\t笹塚駅\t徒歩6分\t\t京王線\t代田橋駅\t徒歩7分\t\t京王線\t明大前駅...,1K,8年6ヶ月,南,22.05m2,12,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t浴室乾燥機\t／\t温水洗浄...,...,0,0,0,0,0,1,0,0,1,0
3,4,150000,東京都杉並区高円寺南２丁目23-2,総武線・中央線（各停）\t高円寺駅\t徒歩9分\t\t丸ノ内線(池袋－荻窪)\t新高円寺駅\...,2LDK,29年4ヶ月,南,60.48m2,3,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t温水洗浄便座／\t洗面台独立,...,0,0,0,0,0,0,0,0,1,0
4,5,74000,東京都葛飾区金町３丁目7-2,京成金町線\t京成金町駅\t徒歩5分\t\t常磐線\t金町(東京都)駅\t徒歩7分\t\t京...,2DK,31年7ヶ月,南,39.66m2,1,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t追焚機能,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31462,31466,80000,東京都板橋区蓮根２丁目,都営三田線\t蓮根駅\t徒歩7分\t\t都営三田線\t西台駅\t徒歩10分\t\t都営三田線...,2DK,30年0ヶ月,南,37.9m2,1,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t追焚機能／\t洗面台独立,...,0,0,0,0,0,0,0,0,1,0
31463,31467,54500,東京都世田谷区太子堂５丁目17-1,東急田園都市線\t三軒茶屋駅\t徒歩6分\t\t東急世田谷線\t西太子堂駅\t徒歩4分\t\...,1R,39年7ヶ月,西,12.04m2,4,専用バス／\t専用トイレ／\tシャワー,...,1,0,0,0,0,0,0,0,1,0
31464,31468,125000,東京都江東区南砂４丁目,東西線\t南砂町駅\t徒歩5分\t\t都営新宿線\t大島(東京都)駅\t徒歩26分\t\t東...,3DK,45年10ヶ月,南,60m2,3,バス・トイレ別／\tシャワー／\t追焚機能／\t温水洗浄便座,...,0,0,0,0,0,0,0,0,1,0
31465,31469,98000,東京都中野区中野２丁目,中央線（快速）\t中野(東京都)駅\t徒歩4分\t\t丸ノ内線(池袋－荻窪)\t新中野駅\t...,1DK,11年0ヶ月,南,29.59m2,1,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t洗面台独立,...,0,0,0,0,0,1,0,0,1,0


In [123]:
train['enviroment']

0        【小学校】 495m\t【大学】 461m\t【小学校】 962m\t【公園】 1103m\...
1                                             【スーパー】 1283m
2        【スーパー】 89m\t【コンビニ】 184m\t【コンビニ】 392m\t【スーパー】 492m
3        【スーパー】 225m\t【スーパー】 448m\t【スーパー】 619m\t【スーパー】 ...
4        【スーパー】 193m\t【スーパー】 298m\t【スーパー】 660m\t【スーパー】 ...
                               ...                        
31462                                                  NaN
31463    【スーパー】 458m\t【スーパー】 540m\t【コンビニ】 131m\t【コンビニ】 ...
31464    【コンビニ】 302m\t【コンビニ】 394m\t【コンビニ】 452m\t【コンビニ】 ...
31465    【スーパー】 485m\t【スーパー】 1051m\t【コンビニ】 476m\t【コンビニ】...
31466                                                  NaN
Name: enviroment, Length: 31467, dtype: object

In [179]:
env = pd.concat([train['enviroment'],test['enviroment']]).reset_index(drop=True)

In [180]:
e_uni = []
for e in env.fillna(''):
    hoge = e.split('\t')
    for h in hoge:
        try:
            e_uni.append(re.search(r'【.+】',h).group()[1:-1])
        except:
            e_uni.append('')

In [181]:
sorted(list(set(e_uni)))

['',
 'クリーニング',
 'コインパーキング',
 'コンビニ',
 'スーパー',
 'デパート',
 'ドラッグストア',
 'レンタルビデオ',
 '公園',
 '図書館',
 '大学',
 '学校',
 '小学校',
 '幼稚園・保育園',
 '月極駐車場',
 '病院',
 '総合病院',
 '郵便局',
 '銀行',
 '飲食店']

In [182]:
env.isnull().sum()

18671

In [219]:
def processing_env(df):
    e_num = []
    dis_ave = []
    dis_min = []

    env = df['enviroment']
    nan_idx = env[env.isnull()]

    for idx in range(len(env)):
        if idx in nan_idx:
            e_num.append(0)
            dis_ave.append(-1)
            dis_min.append(-1)
        else:
            e_split = env[idx].split('\t')
            e_num.append(len(e_split))  

            distance = []
            for s in e_split:
                distance.append(int(re.search(r'\d+',s).group()))
                
            dis_ave.append(np.array(distance).mean())
            dis_min.append(np.array(distance).min())

    print(len(e_num),len(dis_ave))
    df['env_num'] = e_num
    df['dis_ave'] = dis_ave
    df['dis_min'] = dis_min

    df[df['dis_ave']==-1]['dis_ave'] = np.array(dis_ave).mean()
    df[df['dis_min']==-1]['dis_min'] = np.array(dis_min).mean()
    
    return df

In [220]:
def processing_school(df):
    school = []
    univ = []
    
    for e in df['enviroment'].fillna(''):
        if '大学' in e:
            univ.append(1)
        else:
            univ.append(0)
            
        if '学校' in e or '小学校' in e or  '幼稚園・保育園' in e:
            school.append(1)
        else:
            school.append(0)
    
    df['school'] = school
    df['univ'] = univ
    
    return df

In [221]:
train = pd.read_csv('../processed_data/train_v7.csv')
test = pd.read_csv('../processed_data/test_v7.csv')

In [249]:
access = pd.concat([train['access'],test['access']]).reset_index(drop=True)

In [285]:
walk_time = pd.concat([train['walk_time'],test['walk_time']]).reset_index(drop=True)

nearest_st = []
for idx,a in enumerate(access[:]):
    if '徒歩' in a:
        split_a = a.split('\t\t')
        
        st = []
        for way in split_a:
            if '徒歩' in way:
                if '\t' in way:
                    a=1
                elif '\u3000' in way:
                    print(way)
                elif '・' in way:
                    print(way)
                    
#         t_list = [int(i) for i in walk_time[idx][1:-1].split(',')]
#         nearest_st.append(st[t_list.index(min(t_list))])
        

京急本線・大森町駅・徒歩22分
京急本線・大森海岸駅：徒歩5分
京急本線・立会川駅・徒歩12分
京急本線・雑色駅・徒歩15分
京成本線・新三河島駅・徒歩8分
京急空港線・穴守稲荷駅・徒歩14分
京浜東北線・大井町駅・徒歩20分
銀座線・東西線・浅草線　日本橋駅　徒歩5分　/　浅草線　宝町駅　徒歩3分　/　有楽町線　銀座一丁目駅　徒歩10分
丸ノ内線・後楽園駅・徒歩10分
京急本線・大森町駅：徒歩20分
東急多摩川線・下丸子駅・徒歩10分
京急本線・京急蒲田駅：徒歩18分
東急池上線・池上・徒歩23分
京浜東北・根岸線品川駅徒歩8分
京急本線・天空橋駅：徒歩7分
有楽町線・東池袋駅・徒歩10分
京急本線・京急蒲田駅・徒歩15分
京急空港線：糀谷駅・徒歩14分
JR常磐線　三河島駅　徒歩7分
都営浅草線・西馬込駅・徒歩12分
東急多摩川線・矢口渡駅：徒歩15分
京急本線・大森海岸駅：徒歩15分
西武池袋・豊島線椎名町駅徒歩11分
京浜東北・根岸線王子駅バス25分常盤台教会下車徒歩2分
副都心線・雑司ヶ谷駅・徒歩10分
都電荒川線・荒川区役所前駅・徒歩10分
丸ノ内線・新大塚駅・徒歩15分
東武東上線・北池袋駅・徒歩15分
りんかい線・品川シーサイド駅・徒歩14分
東急多摩川線　矢口渡駅徒歩9分
東急多摩川線　武蔵新田駅　徒歩18分
東急多摩川線・矢口渡駅・徒歩12分
京急本線・青物横丁駅・徒歩10分
東急池上線・荏原中延駅・徒歩15分
東急池上線・千鳥町駅・徒歩15分
西武池袋線・椎名町駅・徒歩16分
京急本線・梅屋敷駅・徒歩10分
山手線・品川駅・徒歩20分
京急本線・鮫洲駅・徒歩15分
中央・総武緩行線市ヶ谷駅徒歩10分
東京メトロ日比谷線　南千住　徒歩21分
東急池上線・洗足池駅・徒歩20分
東急池上線・長原駅・徒歩25分
都営浅草線・荏原町駅：徒歩15分
中央・総武緩行線市ヶ谷駅徒歩3分
京急本線・京急蒲田駅・徒歩22分
東急多摩川線・武蔵新田駅・徒歩12分
東急池上線・蓮沼駅・徒歩15分
日暮里舎人ライナー・赤土小学校前駅・徒歩8分
京浜東北線・蒲田駅・徒歩20分
東武伊勢崎線　押上駅　徒歩28分
・東京メトロ銀座線『上野広小路』徒歩8分・山手線『上野』徒歩10分
JR京浜東北線・大森駅：徒歩25分
東急池上線・久が原駅・徒歩11分
京急本線・大森海岸駅：徒歩

In [333]:
hoge = []
a_,b,c,d = 0,0,0,0
for a in access:
    ways = a.split('\t\t')
    for way in ways:
        hun = way.count('分')
        if hun ==1:
            a_ += 1
        elif hun==2:
            b+=1
        elif hun==3:
            c+=1
        else:
            d+=1
            print(way)
#             print(way)
            
#     hoge.append(len(ways))

中央線　中野駅
丸ノ内線　新中野駅
丸ノ内線　中野坂上駅
丸ノ内線　新中野駅
都営大江戸線門前仲町
東急世田谷線　西太子堂駅
丸ノ内方南支線　中野新橋駅
西武新宿線　野方駅
西武新宿線　新井薬師前駅
東急世田谷線　下高井戸駅
東急世田谷線　西太子堂駅
東急東横線　都立大学駅
京王線　笹塚
埼京線　十条駅
東急目黒線大岡山駅
東急世田谷線　世田谷駅
京王線　幡ヶ谷駅
東京メトロ丸の内線　南阿佐ヶ谷駅
丸ノ内方南支線　中野新橋駅
丸ノ内線　新中野駅
日比谷線　中目黒駅
丸ノ内線　中野坂上駅
京王井の頭線　永福町駅
中央線　東中野駅
東京メトロ有楽町線　東池袋駅
都営大江戸線　西新宿五丁目駅
都営三田線：千石駅
京王線　八幡山
★
総武線　大久保駅
小岩
西武新宿線新井薬師前駅
新柴又
丸ノ内方南支線　中野富士見町駅
JR埼京線大崎
落合南長崎駅
西武新宿線　沼袋駅
丸ノ内線新高円寺駅
京王線　千歳烏山駅
東京メトロ丸ノ内方南支線　中野富士見町駅
南北線　西ヶ原駅
JR京浜東北線大森駅
東急東横線祐天寺駅
西武新宿線　高田馬場駅
東急目黒線奥沢駅
竹ノ塚
京王井の頭線　浜田山駅
東急多摩川線　矢口渡駅
日比谷線恵比寿駅
ＪＲ中央線　阿佐ヶ谷駅
京王井の頭線　明大前駅
中央線高円寺駅
西武新宿線　中井駅
京王井の頭線　久我山駅
東京都東武東上線下板橋駅
中央線　中野駅
丸ノ内線　西新宿駅
JR山手線　鶯谷
丸ノ内線　西新宿駅
東急池上線長原
東急東横線学芸大学駅
東武東上線：北池袋駅
東急東横線祐天寺駅
丸ノ内線　西新宿駅
京王井の頭線　久我山駅
京王井の頭線　高井戸駅
京王線　八幡山駅
JR中央線　荻窪駅
東急東横線中目黒駅
京王井の頭線　神泉駅
京王線　笹塚駅
有楽町線　小竹向原駅
丸ノ内線　新高円寺駅
東急田園都市線駒沢大学駅
丸ノ内線　新中野駅
京王井の頭線　井の頭公園駅
有楽町線：東池袋駅
東急東横線代官山駅
東京メトロ南北線　駒込
京王井の頭線　久我山駅
丸の内線　新中野駅
東急東横線中目黒駅
丸ノ内方南支線　中野新橋駅
東京メトロ南北線　東大前駅
丸ノ内線　中野坂上駅
南北線　白金高輪駅
京王井の頭線　高井戸駅
高野
丸ノ内線　東高円寺駅
新井薬師前駅
京王井の頭線　久我山駅
日比谷線中目黒駅
東京メトロ丸の内線　南阿佐ヶ谷駅
ＪＲ中央線　阿佐ヶ谷駅
京浜東北

丸ノ内方南支線　中野富士見町駅
ＪＲ蒲田駅まで自転車で
丸ノ内線　新高円寺駅
山手線　新宿駅
京王井の頭線池ノ上駅
付近にバス停有。
京王井の頭線　東松原駅
東急世田谷線　山下駅
東急大井町線　緑が丘駅
丸ノ内線　中野坂上駅
東急田園都市線　池尻大橋駅
都営大江戸線　西新宿五丁目
都営浅草線新橋
京王井の頭線　久我山
丸ノ内線　中野坂上駅
東武東上線　下赤塚
東急大井町線　尾山台駅
東急東横線学芸大学駅
京王線　千歳烏山駅
ＪＲ中央線　荻窪駅
丸ノ内線　中野坂上駅
中央線　高円寺駅
西武新宿線　野方駅
東急田園都市線　池尻大橋駅
新江古田駅
都営大江戸線　西新宿五丁目
西武池袋線　東長崎駅
東急東横線祐天寺駅
丸ノ内線　中野富士見町


In [330]:
set(hoge)

{2, 3}

In [335]:
a_,b,c,d

(176652, 2591, 3, 1288)